<a href="https://colab.research.google.com/github/Miragecore/MLOps_Coursera/blob/develop/C1/W1/Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

대단한 내용은 없다 서버 만들기 연습인듯

구글 드라이브 마운트

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

#!mkdir -p /drive
#umount /drive
#!mount --bind /content/drive/MyDrive /drive
#!mkdir -p /drive/ngrok-ssh
#!mkdir -p ~/.ssh

구글 드라이브에 폴더 생성  
깃허브에서 구글 워크 드라이브로 requirements.txt 복사해서 환경 구성

In [ ]:
import os

root_path = '/content/drive/MyDrive'
paths = ['MLOps_Coursera', 'ML_for_Production', 'Course1', 'Week1']

os.chdir(root_path)

for folder in paths:
  if not os.path.isdir(folder):
    os.mkdir(folder)
    print('mkdir {folder}')
  os.chdir(folder)

print(os.getcwd())

#강좌에서 제공하는 requirements대로 설치하면 호환성에 문제가 생겨서 Requirements 수정
!wget https://raw.githubusercontent.com/Miragecore/MLOps_Coursera/develop/C1/W1/requirements.txt -O requirements.txt
!pip install -r requirements.txt

In [12]:
from IPython.display import Image, display

필요한 파일은 구글드라이브에 미리 올려둔다.  
/images


In [ ]:
# Some example images
image_files = [
    'apple.jpg',
    'clock.jpg',
    'oranges.jpg',
    'car.jpg'
]

for image_file in image_files:
    print(f"\nDisplaying image: {image_file}")
    display(Image(filename=f"images/{image_file}"))

In [14]:
dir_name = "images_with_boxes"
if not os.path.exists(dir_name):
    os.mkdir(dir_name)

In [15]:

import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox


def detect_and_draw_box(filename, model="yolov3-tiny", confidence=0.5):
    """Detects common objects on an image and creates a new image with bounding boxes.

    Args:
        filename (str): Filename of the image.
        model (str): Either "yolov3" or "yolov3-tiny". Defaults to "yolov3-tiny".
        confidence (float, optional): Desired confidence level. Defaults to 0.5.
    """
    
    # Images are stored under the images/ directory
    img_filepath = f'images/{filename}'
    
    # Read the image into a numpy array
    img = cv2.imread(img_filepath)
    
    # Perform the object detection
    bbox, label, conf = cv.detect_common_objects(img, confidence=confidence, model=model)
    
    # Print current image's filename
    print(f"========================\nImage processed: {filename}\n")
    
    # Print detected objects with confidence level
    for l, c in zip(label, conf):
        print(f"Detected object: {l} with confidence level of {c}\n")
    
    # Create a new image that includes the bounding boxes
    output_image = draw_bbox(img, bbox, label, conf)
    
    # Save the image in the directory images_with_boxes
    cv2.imwrite(f'images_with_boxes/{filename}', output_image)
    
    # Display the image with bounding boxes
    display(Image(f'images_with_boxes/{filename}'))

In [ ]:
for image_file in image_files:
    detect_and_draw_box(image_file)

In [ ]:
detect_and_draw_box("fruits.jpg")

In [ ]:
detect_and_draw_box("fruits.jpg", confidence=0.3)

In [19]:
dir_name = "images_uploaded"
if not os.path.exists(dir_name):
    os.mkdir(dir_name)

In [20]:
import io
import uvicorn
import numpy as np
import nest_asyncio
from enum import Enum
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import StreamingResponse

In [21]:
# Assign an instance of the FastAPI class to the variable "app".
# You will interact with your api using this instance.
app = FastAPI(title='Deploying a ML Model with FastAPI')

# List available models using Enum for convenience. This is useful when the options are pre-defined.
class Model(str, Enum):
    yolov3tiny = "yolov3-tiny"
    yolov3 = "yolov3"


# By using @app.get("/") you are allowing the GET method to work for the / endpoint.
@app.get("/")
def home():
    return "Congratulations! Your API is working as expected. Now head over to http://localhost:8000/docs."


# This endpoint handles all the logic necessary for the object detection to work.
# It requires the desired model and the image in which to perform object detection.
@app.post("/predict") 
def prediction(model: Model, file: UploadFile = File(...)):

    # 1. VALIDATE INPUT FILE
    filename = file.filename
    fileExtension = filename.split(".")[-1] in ("jpg", "jpeg", "png")
    if not fileExtension:
        raise HTTPException(status_code=415, detail="Unsupported file provided.")
    
    # 2. TRANSFORM RAW IMAGE INTO CV2 image
    
    # Read image as a stream of bytes
    image_stream = io.BytesIO(file.file.read())
    
    # Start the stream from the beginning (position zero)
    image_stream.seek(0)

       # Write the stream of bytes into a numpy array
    file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
    
    # Decode the numpy array as an image
    image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    
    
    # 3. RUN OBJECT DETECTION MODEL
    
    # Run object detection
    bbox, label, conf = cv.detect_common_objects(image, model=model)
    
    # Create image that includes bounding boxes and labels
    output_image = draw_bbox(image, bbox, label, conf)
    
    # Save it in a folder within the server
    cv2.imwrite(f'images_uploaded/{filename}', output_image)
    
    
    # 4. STREAM THE RESPONSE BACK TO THE CLIENT
    
    # Open the saved image for reading in binary mode
    file_image = open(f'images_uploaded/{filename}', mode="rb")
    
    # Return the image as a stream specifying media type
    return StreamingResponse(file_image, media_type="image/jpeg")

Colab에서는 로컬 호스트로 접속이 안되니 ngrok을 이용해서 임시 외부 URL로 접속하도록 변경

You can use ngrok to export a port as an external url. Basically, ngrok takes something available/hosted on your localhost and exposes it to the internet with a temporary public URL.
https://stackoverflow.com/questions/63833593/how-to-run-fastapi-uvicorn-in-google-colab

In [ ]:
# Allows the server to be run in this interactive environment
#nest_asyncio.apply()

# Host depends on the setup you selected (docker or virtual env)
#host = "0.0.0.0" if os.getenv("DOCKER-SETUP") else "127.0.0.1"

# Spin up the server!    
#uvicorn.run(app, host=host, port=8000)

import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)
